# Convergence stage 
## step 1: MACE+OPES simulation

Active learning cycle based on OPES metad simulations and DEAL (data-efficient active learning) scheme

Requirements:
* LAMMPS
* MACE
* PLUMED

## Train MACE models

The first step is to train an ensemble (4) of MACE models using the scripts in `config/mace`.

## LAMMPS simulations: MACE+OPES

In [1]:
import os
os.environ['OPENBLAS_NUM_THREADS'] = '1'

In [2]:
import numpy as np
import yaml
import sys
from pathlib import Path
import matplotlib.pyplot as plt

from ase.visualize import view
from ase.io import write

sys.path.append("../../")

from mlputils.build import build_surface_FeCo, add_molecules

### Create initial configuration

$N_2$ molecule adsorbed on top of a FeCo(110) surface (or $2N$ for the products)

In [3]:
adsorbates = ['N','N']

folder = "./"+"-".join(adsorbates)+"/"
Path(folder).mkdir(exist_ok=True)

In [4]:
atoms = build_surface_FeCo(miller_index=(0,1,1),
                            layers=12, 
                            size=(4,3,1),
                            vacuum=15, 
                            fixed_layers=[0,2])

add_molecules(atoms, adsorbates, height=[1.2,1.2], positions = [[0,0],[1.2,2.4]])

write(f'{folder}input.xyz',atoms,format='extxyz')

atoms.wrap(eps=0.1)
view(atoms,viewer='x3d')

ATOMS: 144 (48 fixed - 96 free)
Fixed atoms:  [  1   2   3   4  13  14  15  16  25  26  27  28  37  38  39  40  49  50
  51  52  61  62  63  64  73  74  75  76  85  86  87  88  97  98  99 100
 109 110 111 112 121 122 123 124 133 134 135 136]
Free atoms:  [  5   6   7   8   9  10  11  12  17  18  19  20  21  22  23  24  29  30
  31  32  33  34  35  36  41  42  43  44  45  46  47  48  53  54  55  56
  57  58  59  60  65  66  67  68  69  70  71  72  77  78  79  80  81  82
  83  84  89  90  91  92  93  94  95  96 101 102 103 104 105 106 107 108
 113 114 115 116 117 118 119 120 125 126 127 128 129 130 131 132 137 138
 139 140 141 142 143 144]


/leonardo/pub/userexternal/sperego0/envs/flare_v1.3.3-mkl/lib/python3.8/site-packages/ase/io/extxyz.py:311: UserWarning: Skipping unhashable information adsorbate_info
  warnings.warn('Skipping unhashable information '


### LAMMPS input

Write data

In [5]:
from ase.io.lammpsdata import write_lammps_data

specorder = ['H','N','Fe','Co']

write_lammps_data(f"{folder}/data.lammps", atoms, specorder=specorder , units = 'metal', atom_style='atomic')

MD parameters

In [6]:
# settings
temp = 700
seed = 42 
nsteps = 2000000
tstep = 0.005

# wall along z axis/fixed atoms
wall = 10 + atoms[(atoms.get_atomic_numbers() == 26) | (atoms.get_atomic_numbers() == 27)].get_positions()[:,2].max()
fixed_atoms = atoms.info['fixed']

# path to mace model
mace = 'mace/mace.model'

Write input (We use the template from `configs/lammps`) 

In [7]:
configs_folder = '../../configs/lammps/'

with open(f'{configs_folder}/in.lammps', 'r') as file:
  lmp = file.read()

lmp = lmp.replace('___TEMP___',str(temp))
lmp = lmp.replace('___SEED___',str(seed))
lmp = lmp.replace('___NSTEPS___',str(nsteps))
lmp = lmp.replace('___TSTEP___',str(tstep))
lmp = lmp.replace('___WALL___',str(wall))
lmp = lmp.replace('___FIXED_ATOMS___',fixed_atoms)
lmp = lmp.replace('___MACE___',mace)
lmp = lmp.replace('___SPECORDER___'," ".join(specorder))

with open(f'{folder}in.lammps', 'w') as file:
  file.write(lmp)

print(lmp)

# == SETUP ==
log		    log.lammps append
timer		timeout 86300
atom_style	atomic
units		metal
boundary	p p f
newton		on
atom_modify map yes

# == VARIABLES ==
variable    temp equal 700
variable    seed equal 42
variable    nsteps equal 2000000
variable    tstep equal 0.005
variable    wall equal 20.05424306373336

# == DATA ==
read_data	data.lammps
mass		1 2.016000  #H
mass		2 14.007000 #N
mass		3 55.845000 #Fe
mass		4 58.933194 #Co

group       fixed id 1 2 3 4 13 14 15 16 25 26 27 28 37 38 39 40 49 50 51 52 61 62 63 64 73 74 75 76 85 86 87 88 97 98 99 100 109 110 111 112 121 122 123 124 133 134 135 136
group       free subtract all fixed

# == INTERACTIONS ==
pair_style	mace no_domain_decomposition
pair_coeff	* * mace/mace.model H N Fe Co
timestep	${tstep}

# == MINIMIZATION ==
fix fix_min fixed setforce 0.0 0.0 0.0
minimize	0 0.0001 1000 10000
unfix fix_min

# == OUTPUT ==
thermo		100
thermo_style	custom step temp press ke pe etotal vol pxx pyy pzz pxy pxz pyz
thermo_modify	flush ye

### PLUMED Input

We will use the template from `configs/plumed` and write the index via ASE

In [8]:
configs_folder = '../../configs/plumed/'

with open(f'{configs_folder}/template-opes.dat', 'r') as file:
  plumed = file.read()

# Write atoms index in file
Fe_atoms  = np.argwhere ( atoms.get_atomic_numbers() == 26)[:,0]
Co_atoms  = np.argwhere ( atoms.get_atomic_numbers() == 27)[:,0]
N_atoms   = np.argwhere ( atoms.get_atomic_numbers() == 7)[:,0]
FeCo_atoms= list(Fe_atoms)+list(Co_atoms) 

plumed = plumed.replace('___TEMP___',str(temp))
plumed = plumed.replace('___Fe-ATOMS___',   ",".join( [str(i+1) for i in  Fe_atoms ]))
plumed = plumed.replace('___Co-ATOMS___',   ",".join( [str(i+1) for i in  Co_atoms ]) )
plumed = plumed.replace('___FeCo-ATOMS___', ",".join( [str(i+1) for i in  FeCo_atoms ]))
plumed = plumed.replace('___N-ATOMS___',    ",".join( [str(i+1) for i in  N_atoms ]))
plumed = plumed.replace('___N1-ATOM___',    ",".join( [str(i+1) for i in  [N_atoms[0]] ]))
plumed = plumed.replace('___N2-ATOM___',    ",".join( [str(i+1) for i in  [N_atoms[1]] ]))

# Create two files depending on whether to restart (append to file, but requires KERNELS to exist) or not

plumed_fresh = plumed.replace('___RESTART_MODE___','YES')

with open(f'{folder}plumed.dat', 'w') as file:
  file.write(plumed_fresh)

print(plumed)

# Template file for PLUMED: Opes for 2N -> N2 / FeCo

UNITS LENGTH=A

### GROUPS

N1:   GROUP ATOMS=145
N2:   GROUP ATOMS=146
N:    GROUP ATOMS=145,146
Fe:   GROUP ATOMS=1,3,5,7,9,11,13,15,17,19,21,23,25,27,29,31,33,35,37,39,41,43,45,47,49,51,53,55,57,59,61,63,65,67,69,71,73,75,77,79,81,83,85,87,89,91,93,95,97,99,101,103,105,107,109,111,113,115,117,119,121,123,125,127,129,131,133,135,137,139,141,143
Co:   GROUP ATOMS=2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38,40,42,44,46,48,50,52,54,56,58,60,62,64,66,68,70,72,74,76,78,80,82,84,86,88,90,92,94,96,98,100,102,104,106,108,110,112,114,116,118,120,122,124,126,128,130,132,134,136,138,140,142,144
FeCo: GROUP ATOMS=1,3,5,7,9,11,13,15,17,19,21,23,25,27,29,31,33,35,37,39,41,43,45,47,49,51,53,55,57,59,61,63,65,67,69,71,73,75,77,79,81,83,85,87,89,91,93,95,97,99,101,103,105,107,109,111,113,115,117,119,121,123,125,127,129,131,133,135,137,139,141,143,2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38,40,42,44,46,48,50,52,54,56,58,60,62,64,6

### Run

Run LAMMPS MD with the following command:

> lmp -i in.lammps